In [1]:
cd D:/ML/EVA/JEDi/

D:\ML\EVA\JEDi


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import time
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL

In [4]:
path = ''D:/ML/EVA/JEDi/data/tiny-imagenet-200/''

In [5]:
def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.replace('\n', '').split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])      
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [cv2.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i))) for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(cv2.imread( path + 'val/images/{}'.format(img_name)))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

In [6]:
train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())
train_labels = train_labels.argmax(axis=1)
test_labels = test_labels.argmax(axis=1)

starting loading data
finished loading data, in 86.07648301124573 seconds


In [12]:
bla = cv2.imread(path + 'train/{}/images/{}_{}.JPEG'.format('n02124075', 'n02124075', str(70)))
bla.shape
#D:\ML\EVA\JEDI\data\tiny-imagenet-200\train\n01443537\images

(64, 64, 3)

In [ ]:
print( "train data shape: ",  train_data.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_data.shape )
print( "test_labels.shape: ", test_labels.shape )
#(64,64,3) == (64,64,3)

In [14]:
train_data[0].shape
train_labels[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [16]:
import torch
image = PIL.Image.fromarray(train_data[0]).convert('RGB')
image = np.transpose(image, (2, 0, 1)).astype(np.float32)
image = torch.tensor(image, dtype=torch.float)



In [17]:
image.shape

torch.Size([3, 64, 64])

In [8]:
cd D:/ML/EVA/JEDi/tsai.jedi

D:\ML\EVA\JEDi\tsai.jedi


In [21]:
from datatransforms import train_transform_tinyimagenet,test_transform_tinyimagenet
import config
import torch
kwargs = {'num_workers': config.num_workers, 'pin_memory': config.pin_memory} if config.use_cuda else {}
trainset = train_transform_tinyimagenet(image_list=train_data, label=train_labels)
train_loader_tinyimgnet_s12 = torch.utils.data.DataLoader(trainset,
                                                   batch_size=config.batch_size, shuffle=True, **kwargs)
testset = test_transform_tinyimagenet(image_list=test_data, label=test_labels)
test_loader_tinyimgnet_s12 = torch.utils.data.DataLoader(testset,
                                                  batch_size=config.batch_size, shuffle=True, **kwargs)

In [22]:
del train_data, train_labels, test_data, test_labels, testset, trainset

In [4]:
import sys
sys.path.append('D:/ML/EVA/JEDI/tsai.jedi/Models')
from S9_resnet import resnet18
from S7 import model_summary
import config

In [5]:
model_ = resnet18(num_classes=200, pretrained=False).to(config.device)
#print(model_summary(model_, (3,64,64)))

In [1]:
cd D:/ML/EVA/JEDi/tsai.jedi

D:\ML\EVA\JEDi\tsai.jedi


In [2]:
import config 
config.imagenet_directory = 'D:/ML/EVA/JEDi/data'
import config
config.update_var()

In [3]:
import sys
from lr_finder import LRFinder
import torch.nn as nn
from dataloader import train_loader_tinyimgnet_s12, test_loader_tinyimgnet_s12
from aftereffects import get_image_with_target, plots
import matplotlib.pyplot as plt
import torch.optim as optim
import sys
import config

Files already downloaded and verified
Files already downloaded and verified
starting loading data
finished loading data, in 24.184310913085938 seconds


In [6]:
from torch.optim.lr_scheduler import StepLR, OneCycleLR
from Engine_train_test import train_cyclic, test
import torch.optim as optim

optimizer = optim.SGD(model_.parameters(), lr=9.9E-03, momentum=0.9, weight_decay=5e-4)
scheduler = OneCycleLR(optimizer,max_lr=9.9E-03,epochs = 50,steps_per_epoch = len(train_loader_tinyimgnet_s12),
                       pct_start = 0.21,div_factor = 10,final_div_factor = 1)
valid_acc = []
valid_loss = []
train_acc = []
train_loss = []
for epoch in range(1, 51):
    train_a, train_l = train_cyclic(model_, config.device, train_loader_tinyimgnet_s12, optimizer, epoch,
                             is_cyclicLR = True,scheduler = scheduler,l1_regularization=[0, 1])
    train_acc.append(train_a)
    train_loss.append(train_l)
    valid_a, valid_l = test(model_, config.device, test_loader_tinyimgnet_s12) 
    # Appending to loss and accuracy lists
    valid_acc.append(valid_a)
    train_loss.append(valid_l)

KeyboardInterrupt: 

In [ ]:
torch.save(model_, 'D:/ML/EVA/JEDI/tsai.jedi/model_objects/s12.pt')

In [11]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 3            |        cudaMalloc retries: 3         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    3651 MB |    3652 MB |    4918 MB |    1266 MB |\n|       from large pool |    3647 MB |    3647 MB |    4894 MB |    1246 MB |\n|       from small pool |       4 MB |      19 MB |      24 MB |      19 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |    3651 MB |    3652 MB |    4918 MB |    1266 MB |\n|       from large pool |    3647 MB |    3647 MB |